In [15]:
import mxnet as mx
import numpy as np
import cPickle
import cv2

def extractImagesAndLabels(path, file):
    f = open(path+file, 'rb')
    dict = cPickle.load(f)
    images = dict['data']
    images = np.reshape(images, (10000, 3, 32, 32))
    labels = dict['labels']
    imagearray = mx.nd.array(images)
    labelarray = mx.nd.array(labels)
    return imagearray, labelarray

def extractCategories(path, file):
    f = open(path+file, 'rb')
    dict = cPickle.load(f)
    return dict['label_names']

def saveCifarImage(array, path, file):
    # array is 3x32x32. cv2 needs 32x32x3
    array = array.asnumpy().transpose(1,2,0)
    # array is RGB. cv2 needs BGR
    array = cv2.cvtColor(array, cv2.COLOR_RGB2BGR)
    # save to PNG file
    return cv2.imwrite(path+file+".png", array)

imgarray, lblarray = extractImagesAndLabels("/home/alex/Документы/formxnet/cifar-10-batches-py/", "data_batch_2")
print imgarray.shape
print lblarray.shape

categories = extractCategories("/home/alex/Документы/formxnet/cifar-10-batches-py/", "batches.meta")

cats = []
for i in range(1,20):
    saveCifarImage(imgarray[i], "./", "image"+(str)(i))
    category = lblarray[i].asnumpy()
    category = (int)(category[0])
cats.append(categories[category])
print cats

(10000L, 3L, 32L, 32L)
(10000L,)
['frog']


In [17]:
def buildTrainingSet(path):
    training_data = []
    training_label = []
    for f in ("data_batch_1", "data_batch_2", "data_batch_3", "data_batch_4", "data_batch_5"):
        imgarray, lblarray = extractImagesAndLabels(path, f)
        if not training_data:
            training_data = imgarray
            training_label = lblarray
        else:
            training_data = mx.nd.concatenate([training_data, imgarray])
            training_label = mx.nd.concatenate([training_label, lblarray])
    return training_data, training_label


path="/home/alex/Документы/formxnet/cifar-10-batches-py/"
batch=128

training_data, training_label = buildTrainingSet(path)
train_iter = mx.io.NDArrayIter(
    data=training_data, label=training_label, batch_size=batch, shuffle=True)

valid_data, valid_label = extractImagesAndLabels(path, "test_batch")
valid_iter = mx.io.NDArrayIter(
    data=valid_data, label=valid_label, batch_size=batch, shuffle=True)

print training_data.shape
print training_label.shape
print valid_data.shape
print valid_label.shape

ValueError: The truth value of an NDArray with multiple elements is ambiguous.